In [6]:
# install.packages('languageserver', dependencies = TRUE)
# install.packages('devtools', dependencies = TRUE)
# devtools::install_github('IRkernel/IRkernel',dependencies = TRUE)
# IRkernel::installspec()
# devtools::install_github("mrcieu/gwasglue")
# install_github("jrs95/gassocplot",dependencies=TRUE,upgrade="always")

In [7]:
suppressPackageStartupMessages(suppressWarnings({
    library(gwasglue)
    library(dplyr)
    library(gassocplot)
    library(coloc)
    library(ggplot2)
    library(foreach)
    library(doParallel)
}))
# identify maximum cores available
# ncpus <- detectCores()

# sets how many cores to use
# 2 cores took ~6 min to write gwas hits to file
# 4 cores took ~3 min to write gwas hits to file
registerDoParallel(4)


API: public: http://gwas-api.mrcieu.ac.uk/



In [8]:
# wbc count related to autoimmune disease
gwas_dataset = "ieu-b-30"
dummy_dataset = "ieu-a-7"
gwasinfo(id=as.character(gwas_dataset))
# gwasinfo(id=as.character(dummy_dataset))

id,trait,ontology,unit,sample_size,author,build,mr,category,subcategory,group_name,year,priority,population,consortium,sex,note
<chr>,<chr>,<lgl>,<lgl>,<int>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<lgl>
ieu-b-30,white blood cell count,NA,NA,563946,"Vuckovic, D",HG19/GRCh37,1,Continuous,Immune system,public,2020,0,European,Blood Cell Consortium,Males and Females,NA


In [9]:
# identify top GWAS hits and sort by p-value order (lowest to highest)
top <- ieugwasr::tophits('ieu-b-30') %>% arrange(p)
# str(top)

In [15]:
window_size <- 50000
if (!dir.exists("gwas_hits")) {
    dir.create("gwas_hits")
}

# parallel loop over the top GWAS hits in p-value order and iterate over rows using normal for loop
foreach(i = 1:nrow(top)) %dopar% {
    # str(top[i,])
    # add leading 0s to pval_rank_gwashit so that it always has 3 digits
    pval_rank_gwashit <- sprintf("%03d", i)
    chr_gwashit <- top[i, ]$chr
    pos_gwashit <- top[i, ]$position
    
    # print(paste0("Processing GWAS hit ", pval_rank_gwashit, " of ", nrow(top)))

    lower <- pos_gwashit - window_size
    upper <- pos_gwashit + window_size
    # print(paste(pval_rank_gwashit," ",chr_gwashit," ",lower," ",upper))
    chrpos <- paste0(chr_gwashit, ":", lower, "-", upper)
    # print(chrpos)
    
    # extract SNPS in the region of top hit
    out <- ieugwasr_to_coloc(
        id1 = as.character(gwas_dataset),
        id2 = as.character(dummy_dataset),
        chrompos = as.character(chrpos),
        type1 = "quant",
        # type2 = "cc"
    )

    # get rid of dummy dataset
    out <- out[1]$dataset1
    # str(out)
    
    # export to file (tab-delimited) in directory named gwas_hits in the following format:
    # <pval rank>_<chr>_<lower bound>_<upper bound>.tsv
    write.table(out,
        file = paste0(
            "gwas_hits/", pval_rank_gwashit, "_", chr_gwashit, "_", lower, "_", upper, "_gwas.tsv"
        ),
        sep = "\t", quote = FALSE, row.names = FALSE, col.names = TRUE
    )
}

[[1]]
NULL

[[2]]
NULL

[[3]]
NULL

[[4]]
NULL

[[5]]
NULL

[[6]]
NULL

[[7]]
NULL

[[8]]
NULL

[[9]]
NULL

[[10]]
NULL

[[11]]
NULL

[[12]]
NULL

[[13]]
NULL

[[14]]
NULL

[[15]]
NULL

[[16]]
NULL

[[17]]
NULL

[[18]]
NULL

[[19]]
NULL

[[20]]
NULL

[[21]]
NULL

[[22]]
NULL

[[23]]
NULL

[[24]]
NULL

[[25]]
NULL

[[26]]
NULL

[[27]]
NULL

[[28]]
NULL

[[29]]
NULL

[[30]]
NULL

[[31]]
NULL

[[32]]
NULL

[[33]]
NULL

[[34]]
NULL

[[35]]
NULL

[[36]]
NULL

[[37]]
NULL

[[38]]
NULL

[[39]]
NULL

[[40]]
NULL

[[41]]
NULL

[[42]]
NULL

[[43]]
NULL

[[44]]
NULL

[[45]]
NULL

[[46]]
NULL

[[47]]
NULL

[[48]]
NULL

[[49]]
NULL

[[50]]
NULL

[[51]]
NULL

[[52]]
NULL

[[53]]
NULL

[[54]]
NULL

[[55]]
NULL

[[56]]
NULL

[[57]]
NULL

[[58]]
NULL

[[59]]
NULL

[[60]]
NULL

[[61]]
NULL

[[62]]
NULL

[[63]]
NULL

[[64]]
NULL

[[65]]
NULL

[[66]]
NULL

[[67]]
NULL

[[68]]
NULL

[[69]]
NULL

[[70]]
NULL

[[71]]
NULL

[[72]]
NULL

[[73]]
NULL

[[74]]
NULL

[[75]]
NULL

[[76]]
NULL

[[77]]
NULL

[[78]]
NULL

[[79]]
NULL

[[80]]
NULL

[[81]]
NULL

[[82]]
NULL

[[83]]
NULL

[[84]]
NULL

[[85]]
NULL

[[86]]
NULL

[[87]]
NULL

[[88]]
NULL

[[89]]
NULL

[[90]]
NULL

[[91]]
NULL

[[92]]
NULL

[[93]]
NULL

[[94]]
NULL

[[95]]
NULL

[[96]]
NULL

[[97]]
NULL

[[98]]
NULL

[[99]]
NULL

[[100]]
NULL

[[101]]
NULL

[[102]]
NULL

[[103]]
NULL

[[104]]
NULL

[[105]]
NULL

[[106]]
NULL

[[107]]
NULL

[[108]]
NULL

[[109]]
NULL

[[110]]
NULL

[[111]]
NULL

[[112]]
NULL

[[113]]
NULL

[[114]]
NULL

[[115]]
NULL

[[116]]
NULL

[[117]]
NULL

[[118]]
NULL

[[119]]
NULL

[[120]]
NULL

[[121]]
NULL

[[122]]
NULL

[[123]]
NULL

[[124]]
NULL

[[125]]
NULL

[[126]]
NULL

[[127]]
NULL

[[128]]
NULL

[[129]]
NULL

[[130]]
NULL

[[131]]
NULL

[[132]]
NULL

[[133]]
NULL

[[134]]
NULL

[[135]]
NULL

[[136]]
NULL

[[137]]
NULL

[[138]]
NULL

[[139]]
NULL

[[140]]
NULL

[[141]]
NULL

[[142]]
NULL

[[143]]
NULL

[[144]]
NULL

[[145]]
NULL

[[146]]
NULL

[[147]]
NULL

[[148]]
NULL

[[149]]
NULL

[[150]]
NULL

[[151]]
NULL

[[152]]
NULL

[[153]]
NULL

[[154]]
NULL

[[155]]
NULL

[[156]]
NULL

[[157]]
NULL

[[158]]
NULL

[[159]]
NULL

[[160]]
NULL

[[161]]
NULL

[[162]]
NULL

[[163]]
NULL

[[164]]
NULL

[[165]]
NULL

[[166]]
NULL

[[167]]
NULL

[[168]]
NULL

[[169]]
NULL

[[170]]
NULL

[[171]]
NULL

[[172]]
NULL

[[173]]
NULL

[[174]]
NULL

[[175]]
NULL

[[176]]
NULL

[[177]]
NULL

[[178]]
NULL

[[179]]
NULL

[[180]]
NULL

[[181]]
NULL

[[182]]
NULL

[[183]]
NULL

[[184]]
NULL

[[185]]
NULL

[[186]]
NULL

[[187]]
NULL

[[188]]
NULL

[[189]]
NULL

[[190]]
NULL

[[191]]
NULL

[[192]]
NULL

[[193]]
NULL

[[194]]
NULL

[[195]]
NULL

[[196]]
NULL

[[197]]
NULL

[[198]]
NULL

[[199]]
NULL

[[200]]
NULL

[[201]]
NULL

[[202]]
NULL

[[203]]
NULL

[[204]]
NULL

[[205]]
NULL

[[206]]
NULL

[[207]]
NULL

[[208]]
NULL

[[209]]
NULL

[[210]]
NULL

[[211]]
NULL

[[212]]
NULL

[[213]]
NULL

[[214]]
NULL

[[215]]
NULL

[[216]]
NULL

[[217]]
NULL

[[218]]
NULL

[[219]]
NULL

[[220]]
NULL

[[221]]
NULL

[[222]]
NULL

[[223]]
NULL

[[224]]
NULL

[[225]]
NULL

[[226]]
NULL

[[227]]
NULL

[[228]]
NULL

[[229]]
NULL

[[230]]
NULL

[[231]]
NULL

[[232]]
NULL

[[233]]
NULL

[[234]]
NULL

[[235]]
NULL

[[236]]
NULL

[[237]]
NULL

[[238]]
NULL

[[239]]
NULL

[[240]]
NULL

[[241]]
NULL

[[242]]
NULL

[[243]]
NULL

[[244]]
NULL

[[245]]
NULL

[[246]]
NULL

[[247]]
NULL

[[248]]
NULL

[[249]]
NULL

[[250]]
NULL

[[251]]
NULL

[[252]]
NULL

[[253]]
NULL

[[254]]
NULL

[[255]]
NULL

[[256]]
NULL

[[257]]
NULL

[[258]]
NULL

[[259]]
NULL

[[260]]
NULL

[[261]]
NULL

[[262]]
NULL

[[263]]
NULL

[[264]]
NULL

[[265]]
NULL

[[266]]
NULL

[[267]]
NULL

[[268]]
NULL

[[269]]
NULL

[[270]]
NULL

[[271]]
NULL

[[272]]
NULL

[[273]]
NULL

[[274]]
NULL

[[275]]
NULL

[[276]]
NULL

[[277]]
NULL

[[278]]
NULL

[[279]]
NULL

[[280]]
NULL

[[281]]
NULL

[[282]]
NULL

[[283]]
NULL

[[284]]
NULL

[[285]]
NULL

[[286]]
NULL

[[287]]
NULL

[[288]]
NULL

[[289]]
NULL

[[290]]
NULL

[[291]]
NULL

[[292]]
NULL

[[293]]
NULL

[[294]

In [ ]:
# python3 eqtl_build_db.py
# python3 eqtl_query_db.py

In [80]:
# gather up all the gwas and eqtl files
gwas_file_list <- list.files(path = "gwas_hits", pattern = ".tsv", full.names = TRUE)
eqtl_file_list <- list.files(path = "output_full", pattern = ".tsv", full.names = TRUE)

# str(gwas_file_list)
# str(eqtl_file_list)

# make a directory named output_coloc if it doesn't exist
dir.exists("output_coloc") || dir.create("output_coloc")

for (i in seq_along(gwas_file_list)) {

    # loop over elements in gwas_file_list and eqtl_file_list at the same time
    gwas_file <- gwas_file_list[i]
    eqtl_file <- eqtl_file_list[i]

    gwas_data <- as.list(read.table(file = gwas_file, header = TRUE, sep = "\t"))

    # working around formatting issues
    eqtl_data <- read.table(file = eqtl_file, header = FALSE, sep = "\t", fill = TRUE)
    # drop the last column in eqtl_data - not sure why there is an empty column read in from the file???
    eqtl_data <- eqtl_data[, -ncol(eqtl_data)]
    # rename the columns by first element
    colnames(eqtl_data) <- eqtl_data[1, ]
    # drop the first row
    eqtl_data <- eqtl_data[-1, ]
    # set column data types manually
    eqtl_data$pvalues <- as.numeric(eqtl_data$pvalues)
    eqtl_data$N <- as.integer(eqtl_data$N)
    eqtl_data$MAF <- as.numeric(eqtl_data$MAF)
    eqtl_data$beta <- as.numeric(eqtl_data$beta)
    eqtl_data$varbeta <- as.numeric(eqtl_data$varbeta)
    eqtl_data$type <- as.character(eqtl_data$type)
    eqtl_data$snp <- as.character(eqtl_data$snp)
    eqtl_data$z <- as.numeric(eqtl_data$z)
    eqtl_data$chr <- as.integer(eqtl_data$chr)
    eqtl_data$pos <- as.integer(eqtl_data$pos)
    eqtl_data$id <- as.character(eqtl_data$id)

    eqtl_data <- as.list(eqtl_data)

    # run coloc
    res <- coloc::coloc.abf(gwas_data, eqtl_data)

    #  Get locus from filenames
    SNP_locus_name <- basename(gwas_file)
    SNP_locus_name <- sub("_gwas\\..*$", "", SNP_locus_name)
    # split on _ assign first element to pval_rank, second to chr, third to lower bound, fourth to upper bound
    SNP_locus_name <- strsplit(SNP_locus_name, "_")
    SNP_locus_name <- unlist(SNP_locus_name)
    pval_rank <- SNP_locus_name[1]
    chr <- SNP_locus_name[2]
    lower <- SNP_locus_name[3]
    upper <- SNP_locus_name[4]


    # write output to separate log files (if as.numeric(res$summary["PP.H4.abf"]) > 0.8), tab-separated
    if (as.numeric(res$summary["PP.H4.abf"]) >= 0.8) {
        write.table(res$summary,
            file = paste0(
                "output_coloc/H4gt80.tsv"
            ),
            sep = "\t", quote = FALSE, row.names = FALSE, col.names = TRUE, append = TRUE
        )
    } else if (as.numeric(res$summary["PP.H4.abf"]) < 0.8) {
        write.table(res$summary,
            file = paste0(
                "output_coloc/H4lt80.tsv"
            ),
            sep = "\t", quote = FALSE, row.names = FALSE, col.names = TRUE, append = TRUE
        )
    }
    # write.table(paste(pval_rank,chr,lower,upper,as.numeric(res$summary["PP.H1.abf"]),as.numeric(res$summary["PP.H2.abf"]),as.numeric(res$summary["PP.H3.abf"]),as.numeric(res$summary["PP.H4.abf"])), file = paste0("output_coloc/", "posteriors.tsv"), sep = "\t", quote=FALSE, row.names = FALSE, col.names = FALSE, append = TRUE)
}


[1] TRUE

Warning message in if (!(d$type %in% c("quant", "cc"))) stop("dataset ", suffix, :
“the condition has length > 1 and only the first element will be used”
Warning message in if (!(d$type %in% c("quant", "cc"))) stop("dataset ", suffix, :
“the condition has length > 1 and only the first element will be used”
Warning message in if (!(d$type %in% c("quant", "cc"))) stop("dataset ", suffix, :
“the condition has length > 1 and only the first element will be used”
Warning message in if (d$type == "cc" & "pvalues" %in% nd) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (d$type == "quant") {:
“the condition has length > 1 and only the first element will be used”
Warning message in sdY.est(d$varbeta, d$MAF, d$N):
“estimating sdY from maf and varbeta, please directly supply sdY if known”
Warning message in if (type == "quant") {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (!(d$type %in% c("quant", "

PP.H0.abf PP.H1.abf PP.H2.abf PP.H3.abf PP.H4.abf 
3.14e-213  8.83e-01 1.03e-214  2.89e-02  8.83e-02 
[1] "PP abf for shared variant: 8.83%"


Warning message in write.table(res$summary, file = paste0("output_coloc/H4lt80.tsv"), :
“appending column names to file”
Warning message in if (!(d$type %in% c("quant", "cc"))) stop("dataset ", suffix, :
“the condition has length > 1 and only the first element will be used”
Warning message in if (!(d$type %in% c("quant", "cc"))) stop("dataset ", suffix, :
“the condition has length > 1 and only the first element will be used”
Warning message in if (!(d$type %in% c("quant", "cc"))) stop("dataset ", suffix, :
“the condition has length > 1 and only the first element will be used”
Warning message in if (d$type == "cc" & "pvalues" %in% nd) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (d$type == "quant") {:
“the condition has length > 1 and only the first element will be used”
Warning message in sdY.est(d$varbeta, d$MAF, d$N):
“estimating sdY from maf and varbeta, please directly supply sdY if known”
Warning message in if (type == "quant") {:

PP.H0.abf PP.H1.abf PP.H2.abf PP.H3.abf PP.H4.abf 
   0.0000    0.8870    0.0000    0.0238    0.0887 
[1] "PP abf for shared variant: 8.87%"


...

Warning message in write.table(res$summary, file = paste0("output_coloc/H4lt80.tsv"), :
“appending column names to file”
Warning message in if (!(d$type %in% c("quant", "cc"))) stop("dataset ", suffix, :
“the condition has length > 1 and only the first element will be used”
Warning message in if (!(d$type %in% c("quant", "cc"))) stop("dataset ", suffix, :
“the condition has length > 1 and only the first element will be used”
Warning message in if (!(d$type %in% c("quant", "cc"))) stop("dataset ", suffix, :
“the condition has length > 1 and only the first element will be used”
Warning message in if (d$type == "cc" & "pvalues" %in% nd) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (d$type == "quant") {:
“the condition has length > 1 and only the first element will be used”
Warning message in sdY.est(d$varbeta, d$MAF, d$N):
“estimating sdY from maf and varbeta, please directly supply sdY if known”
Warning message in if (type == "quant") {:

PP.H0.abf PP.H1.abf PP.H2.abf PP.H3.abf PP.H4.abf 
 2.99e-16  8.90e-01  7.09e-18  2.10e-02  8.90e-02 
[1] "PP abf for shared variant: 8.9%"


Warning message in write.table(res$summary, file = paste0("output_coloc/H4lt80.tsv"), :
“appending column names to file”
Warning message in if (!(d$type %in% c("quant", "cc"))) stop("dataset ", suffix, :
“the condition has length > 1 and only the first element will be used”
Warning message in if (!(d$type %in% c("quant", "cc"))) stop("dataset ", suffix, :
“the condition has length > 1 and only the first element will be used”
Warning message in if (!(d$type %in% c("quant", "cc"))) stop("dataset ", suffix, :
“the condition has length > 1 and only the first element will be used”
Warning message in if (d$type == "cc" & "pvalues" %in% nd) {:
“the condition has length > 1 and only the first element will be used”
Warning message in if (d$type == "quant") {:
“the condition has length > 1 and only the first element will be used”
Warning message in sdY.est(d$varbeta, d$MAF, d$N):
“estimating sdY from maf and varbeta, please directly supply sdY if known”
Warning message in if (type == "quant") {:

PP.H0.abf PP.H1.abf PP.H2.abf PP.H3.abf PP.H4.abf 
 0.001960  0.838000  0.000192  0.081800  0.077800 
[1] "PP abf for shared variant: 7.78%"


Warning message in write.table(res$summary, file = paste0("output_coloc/H4lt80.tsv"), :
“appending column names to file”
Warning message in if (!(d$type %in% c("quant", "cc"))) stop("dataset ", suffix, :
“the condition has length > 1 and only the first element will be used”


In [ ]:
# manually choose individual files
gwas_file <- "gwas_hits/001_6_31186115_31286115_gwas.tsv"
gwas_data <- as.list(read.table(file=gwas_file, header = TRUE, sep = '\t'))

# print  data types of each column
# sapply(gwas_data, class)

# working around formatting issues
eqtl_file <- "output_full/001_6_31186115_31286115_eQTLs.tsv"
eqtl_data <- read.table(file=eqtl_file, header = FALSE, sep = '\t',fill = TRUE)
# drop the last column in eqtl_data (not sure why there is an empty column read in from the file)
eqtl_data <- eqtl_data[,-ncol(eqtl_data)]
# rename the columns by first element
colnames(eqtl_data) <- eqtl_data[1,]
# drop the first row
eqtl_data <- eqtl_data[-1,]
# set column data types manually
eqtl_data$pvalues <- as.numeric(eqtl_data$pvalues)
eqtl_data$N <- as.integer(eqtl_data$N)
eqtl_data$MAF <- as.numeric(eqtl_data$MAF)
eqtl_data$beta <- as.numeric(eqtl_data$beta)
eqtl_data$varbeta <- as.numeric(eqtl_data$varbeta)
eqtl_data$type <- as.character(eqtl_data$type)
eqtl_data$snp <- as.character(eqtl_data$snp)
eqtl_data$z <- as.numeric(eqtl_data$z)
eqtl_data$chr <- as.integer(eqtl_data$chr)
eqtl_data$pos <- as.integer(eqtl_data$pos)
eqtl_data$id <- as.character(eqtl_data$id)

eqtl_data <- as.list(eqtl_data)

# final sanity check
# str(gwas_data)
# str(eqtl_data)

# run coloc 
res <- coloc::coloc.abf(gwas_data, eqtl_data) 

In [ ]:
# obsolete (delete this later)

# get list of all files in output directory
# files <- list.files(path = "output", pattern = ".*.tsv", full.names = TRUE)

# truncate filelist to get a tiny subset
# files <- files[1:4]
# files

# This may need to be modified so it's not just taking file names in alphabetical order, but in descending pval order.
for (i in seq(1, length(files), 2)) {
    print(paste("starting",i,files[i],files[i+1]))

    # read in files
    eqtl <- as.list(read.table(file=files[i], header = TRUE, sep = '\t'))
    gwas <- as.list(read.table(file=files[i+1], header = TRUE, sep = '\t'))
    
    # run coloc and write the output to a log file
    res <- coloc::coloc.abf(gwas, eqtl) 
    
    # # make a directory named output_coloc if it doesn't exist
    dir.exists("output_coloc") || dir.create("output_coloc")

    # print numeric value of element of res$summary named PP.H4.abf and filename without extension and without _eQTLS
    # str(res)
    # print(as.numeric(res$summary["PP.H4.abf"]))
    #  remove file extension from files[i] and store in a variable
    SNP_locus_name <- basename(files[i])
    SNP_locus_name <- sub("_eQTLs\\..*$", "", SNP_locus_name)
    SNP_locus_name <- sub("^region_", "", SNP_locus_name)
    
    # write output to a log file, tab-separated
    write.table(paste(SNP_locus_name,as.numeric(res$summary["PP.H1.abf"]),as.numeric(res$summary["PP.H2.abf"]),as.numeric(res$summary["PP.H3.abf"]),as.numeric(res$summary["PP.H4.abf"])), file = paste0("output_coloc/", "PP.H1-4.abf.tsv"), sep = "\t", quote=FALSE, row.names = FALSE, col.names = FALSE, append = TRUE)
    

    # print(paste("eQTL path: ", files[i]))
    # print(paste("GWAS path: ", files[i+1]))
    # create a new list where eqtl and gwas are its elements
    out <- list(eqtl, gwas)

    print(paste("ending",i,files[i],files[i+1]))
}

In [ ]:
# asr::tophits('ieu-a-300') %>% arrange(p)   
# top 
# chrpos <- paste0(top$chr[1], ":", top$position[1] - 90000, "-", top$position[1] + 90000)
# out <- ieugwasr_to_coloc(id1='ieu-a-300', id2='ieu-a-7', chrompos=chrpos)
# res <- coloc::coloc.abf(out[[1]], out[[2]])
# temp <- coloc_to_gassocplot(out)
# gassocplot::stack_assoc_plot(temp$markers, temp$z, temp$corr, traits=temp$traits)

# download.file("https://gwas.mrcieu.ac.uk/files/ieu-a-300/ieu-a-300.vcf.gz","ieu-a-300.vcf.gz")
# download.file("https://gwas.mrcieu.ac.uk/files/ieu-a-300/ieu-a-300.vcf.gz.tbi","ieu-a-300.vcf.gz.tbi")
# download.file("https://gwas.mrcieu.ac.uk/files/ieu-a-7/ieu-a-7.vcf.gz","ieu-a-7.vcf.gz")
# download.file("https://gwas.mrcieu.ac.uk/files/ieu-a-7/ieu-a-7.vcf.gz.tbi","ieu-a-7.vcf.gz.tbi")

# chrpos <- "19:11112306-11292306"
# vout <- gwasvcf_to_coloc("ieu-a-300.vcf.gz", "ieu-a-7.vcf.gz", chrpos)
# vres <- coloc::coloc.abf(vout[[1]], vout[[2]])

# library(gassocplot)
# temp <- coloc_to_gassocplot(vout)
# gassocplot::stack_assoc_plot(temp$markers, temp$z, temp$corr, traits=temp$traits)